# MyHDL 'v2we': Front End Changes

These are some crucial changes to the 'legacy' MyHDL syntax. In general, 'next generation' syntax will force you to follow a little different way of thinking.

To outline the paradigm shifts:

* Instead of trying to convert, enforce library concepts:
  * Re-use verified V* HDL code by thin wrappers
  * Inline-generate only where necessary
* HDL is 'transpiled' (executed), not 'translated' (by AST-transformation)
* Remember: conversion and implementation rules are buried in the extension objects (`@blackbox`es, custom signals, etc.)
* Stricter interface rules, no implicit handling of mixed type parameters (constants, signals)

TOC:

1. [Interface](#Interface)
0.  [Functions](#Functions)
0.  [Signals](#Signals)
    1.  [Shadow signals](#Shadow-signals)

0.  [Variables](#Variables)
0.  [Member assignment](#Member-assignment)
0.  [Class signals](#Class-signals)
0.  [Unsupported constructs](#Other-unsupported-constructs)

In [1]:
from myirl.emulation.myhdl import *

## Interface

Type annotation helps to determine input and output types and are in most cases mandatory in order to create a sane interface. It is necessary to determine which signal is driven and which is consumed within the hierarchy. Currently no automated checking is done due to performance reasons (this may become an option).

You may still pass down either a constant or a signal via a classic port parameter and omit the annotation ('legacy' myHDL style). However, this can have unwanted side effects. For example, it may require you to explicitely `.insert()` a signal into its parenting function, if it ends up undefined in the resulting HDL during conversion. This behaviour may change in later releases.

For the code below, `s` can only be a `bool()` or Signal of it. To avoid warnings, we don't specify a type annotation for `s`.

In [2]:
@block
def unit(a : Signal, q : Signal.Output, s):
    @always_comb
    def worker():
        if s: # This is allowed with a bool or bit signal
            q.next = a
        else:
            q.next = ~a
    
    return locals()

Note also the new `locals()` call instead of `interfaces()` (which is in fact now identical within a Python context). This allows some more analysis (signals, etc.) in the future. It is **required** to use the `locals()` notation within a cythonizeable module.

In [3]:
from myirl.test.common_test import run_ghdl, SYSTEM_DIR

def test(MODE = 0):
    a, q = [ Signal(bool())  for i in range(2) ]
    if MODE:
        s = Signal(bool())
    else:
        s = False
    
    inst = unit(a, q, s)
    return inst

inst = test()
f = inst.elab(targets.VHDL)
tmp0 = unit.ctx.path_prefix

 Insert unit unit_s1_s1_0 
 DEBUG: Writing 'unit' to file /tmp/unit.vhdl 
Finished _elab in 0.0009 secs


### Differences in VHDL output

We output both scenarios and run the resulting VHDL code through analysis plus `diff` to see how the interface is inferred:

In [4]:
run_ghdl(f, inst, debug = False)

0

In [5]:
inst = test(1)
f = inst.elab(targets.VHDL)
run_ghdl(f, inst)

tmp1 = unit.ctx.path_prefix

 Module top_unit: Existing implementation unit, rename to unit_1 
 Insert unit unit_s1_s1_s1 
 DEBUG: Writing 'unit_1' to file /tmp/unit_1.vhdl 
Finished _elab in 0.0009 secs


In [6]:
! diff --label unit --label unit_1 --color=always -u {tmp0}/unit.vhdl {tmp1}/unit_1.vhdl

--- unit
+++ unit_1
@@ -11,22 +11,23 @@
 use work.txt_util.all;
 use work.myirl_conversion.all;
 
-entity unit is
+entity unit_1 is
     port (
         a : in std_ulogic;
-        q : out std_ulogic
+        q : out std_ulogic;
+        s : in std_ulogic
     );
-end entity unit;
+end entity unit_1;
 
-architecture MyIRL of unit is
+architecture MyIRL of unit_1 is
     -- Local type declarations
     -- Signal declarations
 begin
     
 worker:
-    process(a)
+    process(a, s)
     begin
-        if FALSE then
+        if (s = '1') then
             q <= a;
         else
             q <= not a;


### Interface types

Only `Signal` type derivatives will infer to a port. All others pass as 'run time' parameters, unless they are explicitely passed as parameter, specified past the PEP570 `*` delimiter. This has the following effects:

* Bare class constructs don't pass the hierarchy over several layers (use BulkSignals)
* A bare class unrolls into members and in/out direction is guessed
* `dict` or `tuple` container types of signals will create local instances of their children

In [7]:
@block
def unit_local_inst(d, t):
    
    a = d['a']
    b = d['b']
    
    q = t[0]
    
    s = concat(a, b)

    @always_comb
    def worker():
        q.next = s
    
    return instances()

In [8]:
def convert(uut):
    a, b = [ Signal(intbv(0x5)[4:], name=n) for n in ('a', 'b') ]
    q = Signal(intbv()[8:], name='q')
    inst = uut({'a' : a, 'b' : b}, (q,))
    inst.elab(myirl.targets.VHDL)
convert(unit_local_inst)

 Insert unit unit_local_inst__dict__tuple 
 DEBUG: Writing 'unit_local_inst' to file /tmp/unit_local_inst.vhdl 
Finished _elab in 0.0007 secs


/home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/kernel/components.py:481: UserWarning: Dictionaries/Tuples do not add to the interface.
Local signal instances will be created. Use BulkSignal classes instead.
  base.warnings.warn("""Dictionaries/Tuples do not add to the interface.


In [9]:
! grep -A 10 declarations {unit_local_inst.ctx.path_prefix}/unit_local_inst.vhdl

    -- Local type declarations
    -- Signal declarations
    signal q : unsigned(7 downto 0);
    signal a : unsigned(3 downto 0);
    signal b : unsigned(3 downto 0);
begin
    
worker:
    process(a, b)
    begin
        q <= (a & b);
    end process;


### Inferred port signal types

The hierarchical interfaces are restricted to signals with a well defined width, as if they were physical pins.

Therefore, the following data types can not be used as `in`/`out` signal arguments:

* Enum types
* Vector types
* Other custom types not derived from the `Signal` class.

### Type checking

Currently, the type checking is lazy, meaning that only a warning is emitted for built-in signals.
For custom signals, the check is customized, i.e. errors can be raised.

## Variables

This has become stricter: First time variable assignment determines the type. In myHDL, this also counts, however it was possible to assign an expression first time. This is no longer possible.

In [10]:
@block
def tb_var():
    @instance
    def seq():
        v = intbv(10)[5:] # Intbv type
        z = -1             # Unlimited integer
        s = "LUT"
        
    return instances()

Conversion to VHDL:

In [11]:
def convert(uut):
    inst = uut()
    inst.elab(targets.VHDL)
    return inst

convert(tb_var)

Creating sequential 'tb_var/seq' 
 Insert unit tb_var 
 DEBUG: Writing 'tb_var' to file /tmp/tb_var.vhdl 
Finished _elab in 0.0007 secs


[Instance tb_var I/F: [// ID: tb_var_0 to tb_var]]

And resulting output:

In [12]:
!grep -A 10 process {tb_var.ctx.path_prefix}tb_var.vhdl

    process
    variable v : unsigned(4 downto 0);
    variable z : integer;
    variable s : string;
    begin
        v := "01010";
        z := -1;
        s := "LUT";
        wait;
    end process;
end architecture MyIRL;



## Signals

The `components.Signal` class emulates the myHDL signal with the native `intbv` wire type behaviour up to a few corner cases which are handled differently (in particular with arithmetics).

However, there are new signal types introduced:
* ClkSignal: A dedicated clock signal

From the top level, a clock signal will require redefining to `ClkSignal()`.

### Shadow signals

Slicing `ShadowSignal` instances are no longer necessary and are considered deprecated. Currently, they are emulated with a warning.

In [13]:
a = Signal(intbv(0xaa)[8:])
sa = a(4,0)

/home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/emulation/signals.py:12: DeprecatedWarning: Deprecated indexing syntax, use `[]` instead
  warnings.warn(*args, category)


In [14]:
sa.evaluate()

intbv(10)

**Note**: ShadowSignal emulations will not work fully throughout the hierarchy and are for now supported on local signals only

### Concatenation

The `concat()` function can be used in the behavioural as well as the structural section of the `@block`.

In [15]:
@block
def unit(a, b, q, MODE=0):
    
    if MODE:
        s = concat(a, b)
    else:
        s = ConcatSignal(a, b)
    
    @always_comb
    def worker():
        q.next = s
    
    return instances()

In [16]:
def convert(uut):
    a, b = [ Signal(intbv(0x5)[4:], name=n) for n in ('a', 'b') ]
    q = Signal(intbv()[8:], name='q')
    inst = uut(a, b, q, MODE=0) # Change mode to '1' to make the warning disappear
    return inst

inst = convert(unit)
inst.elab(myirl.targets.VHDL)

 Insert unit unit_s4_s4_s8_0 
 DEBUG: Writing 'unit' to file /tmp/unit.vhdl 
Finished _elab in 0.0009 secs


/home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/emulation/signals.py:12: DeprecatedWarning: Deprecated ConcatSignal() syntax, use concat() instead
  warnings.warn(*args, category)


['/tmp/unit.vhdl']

The resulting process in VHDL:

In [17]:
! grep -A 5 worker {unit.ctx.path_prefix}/unit.vhdl

worker:
    process(a, b)
    begin
        q <= (a & b);
    end process;



# Functions

Functions are no longer translated to HDL, but executed. Therefore, you'll have to implement your previous function as a `@blackbox` or `@function` entity if you wish to specifically generate hardware, or transfer the functionality into a HDL block. Very simple operators can be also implemented by derival of `base.BinOp`, etc.

Example of a `reduceOr` pseudo function that can resolve into different implementations, depending on the HDL target:

In [18]:
from myirl.library.reduce import reduceOr

@block
def unit1(a : Signal, q : Signal.Output):
    @always_comb
    def assign():
        q.next = reduceOr(a)
        
    return instances()

VHDL target: REGISTERING [Component 'reduce_xor/reduce_xor']
VHDL target: REGISTERING [Component 'reduce_or/reduce_or']
VHDL target: REGISTERING [Component 'reduce_and/reduce_and']


In [19]:
from myirl.wire import SLV
def convert(uut):
    a = Signal(SLV("1010100"))
    q = Signal(bool())
    inst = uut(a, q)
    return inst

inst = convert(unit1)
inst.elab(myirl.targets.VHDL)

 Insert unit unit1_s7_s1 
 DEBUG Inline blackbox [Component 'reduce_or/reduce_or'] 
 DEBUG: Writing 'unit1' to file /tmp/unit1.vhdl 
DEBUG Skipped [Component 'reduce_or/reduce_or']
Finished _elab in 0.0012 secs


['/tmp/unit1.vhdl']

In [20]:
! grep -A 8 begin {unit1.ctx.path_prefix}unit1.vhdl

begin
    
assign:
    process(a)
    begin
        q <= or_reduce(a);
    end process;

end architecture MyIRL;



## Member assignment

Constructs like `s.next[i] = val` are no longer supported, but can be emulated using variables as shown below. (This particular example is somewhat bloated, as the `v` elements are used nowhere else)

In [21]:
@block
def unit_swap_bits(a : Signal, q : Signal.Output):
    @always_comb
    def worker():
        v = intbv()[2:]

        v[0] = a[1]
        v[1] = a[0]
        
        q.next = v
    
    return instances()

@block
def top(uut):
    a, b = [ Signal(intbv()[2:], name=n) for n in ('a', 'b') ]
    
    inst = uut(a, b)
    return instances()

In [22]:
def test_top(uut):
    inst = top(uut)
    f = inst.elab(targets.VHDL, elab_all=True)
    run_ghdl(f, inst, analyze_only = True)
test_top(unit_swap_bits)

 Insert unit unit_swap_bits_s2_s2 
 Insert unit top__wrapped_wrapper 
 DEBUG: Writing 'unit_swap_bits' to file /tmp/unit_swap_bits.vhdl 
Finished _elab in 0.0008 secs
 DEBUG: Writing 'top' to file /tmp/top.vhdl 
Finished _elab in 0.0154 secs
DEBUG Creating library file /tmp/module_defs.vhdl


In [23]:
! grep -A 8 worker {top.ctx.path_prefix}unit_swap_bits.vhdl

worker:
    process
        variable v : unsigned(1 downto 0);
    begin
        v := "00";
        v(0) := a(1);
        v(1) := a(0);
        q <= v;
    end process;


A nicer/better coding style would be the snippet below (assuming the single bit `t` array signals would be used somewhere else, if not, we'd get by without them using `BLOAT = False`). 
**Note**: concat arguments are MSB..LSB order (which does the swapping the elegant way)

In [24]:
@block
def unit_swap_bits1(a : Signal, q : Signal.Output, BLOAT = True):
    if BLOAT:
        t = [ Signal(bool()) for i in range(2) ]

        a0 = t[0].wireup(a[0]) # Create wire instance
        a1 = t[1].wireup(a[1])
        wiring = q.wireup(concat(*t)) # Note: this is in fact `concat(t[0], t[1])
    else:
        wiring = q.wireup(concat(a[0], a[1]))


    return instances()

In [25]:
test_top(unit_swap_bits1)

 Module top_top: Existing implementation top, rename to top_1 
Using default for BLOAT: True
 BLOAT: use default True 
 BLOAT: use default True 
 Insert unit unit_swap_bits1_s2_s2_1 
 Insert unit top__wrapped_wrapper 
 DEBUG: Writing 'unit_swap_bits1' to file /tmp/unit_swap_bits1.vhdl 
Finished _elab in 0.0014 secs
 DEBUG: Writing 'top_1' to file /tmp/top_1.vhdl 
Finished _elab in 0.0008 secs
DEBUG Creating library file /tmp/module_defs.vhdl


In [26]:
! grep -A 4 begin {top.ctx.path_prefix}unit_swap_bits1.vhdl

begin
    t0 <= a(0);
    t1 <= a(1);
    q <= (t0 & t1);
end architecture MyIRL;


## Class signals

Bare classes are no longer supported, as the use cases are endlessly complex.

It is very much recommended to use [Bulk Signals](bulksignals.ipynb) instead, as it's unlikely that support for nested hierarchies will be added soon.

See also [Classes](classes.ipynb) for internal details.
Basically, you can wrap a class using the `@bulkwrapper` construct, taking the supported target classes as argument:

In [27]:
@bulkwrapper(targets.vhdl)
class OnewaySignalContainer:
    def __init__(self):
        self.a, self.b = [ Signal(bool()) for _ in range(2) ]

In [28]:
@block
def unit(c : OnewaySignalContainer, a : Signal.Output, b : Signal.Output):
    @always_comb
    def worker():
        b.next = c.a
        a.next = c.b
    return instances()

@block
def tb():
    b0, b1 = [ Signal(bool()) for _ in range(2) ]
    c = OnewaySignalContainer()

    inst = unit(c, b0, b1)
    return instances()

def test():
    t = tb()
    t.elab(targets.VHDL, elab_all = True)
    
test()

VHDL target: REGISTERING <class 'myirl.library.bulksignals.OnewaySignalContainer'>
 Insert unit unita_1_sb_1_s1_s1 
 Insert unit tb 
 DEBUG: Writing 'unit' to file /tmp/unit.vhdl 
Extension class uses no lib: c
Finished _elab in 0.0013 secs
 DEBUG: Writing 'tb' to file /tmp/tb.vhdl 
Extension class uses no lib: <class 'myirl.library.bulksignals.OnewaySignalContainer'>
Finished _elab in 0.0008 secs
DEBUG Creating library file /tmp/module_defs.vhdl


In [29]:
!grep -A 10 worker {tb.ctx.path_prefix}/unit.vhdl

worker:
    process(c.a, c.b)
    begin
        b <= c.a;
        a <= c.b;
    end process;

end architecture MyIRL;



## Other unsupported constructs

These should raise early exceptions:

### Augmented assign to list of signals

This one will fail early during AST analysis when using the augmented assign variant below:

In [30]:
@block
def kaputt():
    worker = [ intbv(0)[1:] ]
    # worker += [ Signal(intbv(i)[2:]) for i in range(3) ]
    worker = worker + [ Signal(intbv(i)[2:]) for i in range(3) ]
    return locals()